# Workflow #5

Comparative assessment: Distance variants

In [ ]:
import algo.helper as h
import os.path
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt


# settings
aoi_names = ["at_wien", "at_zs", "at_ib", "at_no", "at_zw", "at_graz_15"]

recalc = False
generate_plots = True
plot_dir = os.path.join("plots", "centr_rdist")

file_suffix = ""

dir_data = "data"

mode = "bike_incwalk"
# tolerable access is determined by input network: all segments that have an index value assigned 
# (other than NULL, > 0) but have mode access set to False
access = "bicycle" 

# plot settings
P_SIZE_S = (4,3)
# colors: distance variants
P_C_D2 = "#2C8DBC"
P_C_D4 = "#056390"
P_C_D7 = "#003650"
# colors: min, mean, max
P_C_MIN = "#00A246"
P_C_MEAN = "#270D9D"
P_C_MAX = "#C60049"

# generated params
file = os.path.join(dir_data, f"r_<aoi_name>_edges{file_suffix}.gpkg")
f_diffstats_dist_cutoff = os.path.join(dir_data, f"centr_diffstats_dist_cutoff.csv")


In [ ]:
compare_dist_refcols = ["centr_sbc_c2000_bp_d4_sum", "centr_sbc_c4000_bp_d4_sum", "centr_sbc_c7000_bp_d4_sum",
                        "centr_sbc_c2000_sp_sum", "centr_sbc_c4000_sp_sum", "centr_sbc_c7000_sp_sum"]

In [ ]:
def run_comparison(aoi, centr_summary, plot_core_aoi=False, plot_core_aoi_mask=True, lw=0.3):
    fn = file.replace("<aoi_name>", aoi)
    if not os.path.exists(fn):
        print(f"ERR: file '{fn}' not found.")
        return
    dir_detail_plot = os.path.join("plots", aoi)
    aoi_core_extent = h.get_aoi_extent(aoi)
    if aoi_core_extent.iloc[0].geometry.area < 1:
        print(f"WARN: invalid core area of '{aoi}' - maybe your input area is too small? Please check input. Skipping.")
        return

    # read file (clipped to bounding box of core extent - needs to be clipped to exact extent after import)
    centr_df = gpd.read_file(fn, bbox=aoi_core_extent.iloc[0].geometry.bounds, engine='pyogrio') # pot. speedup: use arrow
    print("loaded input gdf.", len(centr_df), "edges")
    print("clipping to core AOI extent...")
    centr_df = centr_df.clip(aoi_core_extent)
    print("done.", len(centr_df), "edges")
    cols = centr_df.columns
    print(cols)
    for refcol in compare_dist_refcols:
        cref = h.CentralityDef.from_str(refcol)
        print("processing ref col:", cref)
        # find compare target cols
        for col in cols:
            if not col.startswith("centr_") or not col.endswith("_sum"):
                continue
            cref_label = "" 
            c_label = ""
            c = None
            cand = h.CentralityDef.from_str(col)
            if cref.to_str() == cand.to_str():
                continue
            if cand.type == cref.type and cand.is_bp == cref.is_bp and cand.dfac == cref.dfac and cand.nws == cref.nws and cand.refnet == cref.refnet:
                if cand.decay_from == cref.decay_from and cand.cut > cref.cut:
                    # dist cutoff variant
                    c = cand
                    if cref.decay_from < 0:
                        cref_label = f"{cref.cut / 1000:.0f} km"
                        c_label = f"{c.cut / 1000:.0f} km"
                    else:
                        cref_label = f"{cref.decay_from / 1000:.0f}-{cref.cut / 1000:.0f} km"
                        c_label = f"{c.decay_from / 1000:.0f}-{c.cut / 1000:.0f} km"
                elif cref.decay_from == -1 and not (cand.decay_from > cref.cut or cand.cut < cref.cut):
                    # non-decay as ref. and cref_cut is within the decay range of cand
                    c = cand
                    cref_label = f"{cref.cut / 1000:.0f} km"
                    c_label = f"{c.decay_from / 1000:.0f}-{c.cut / 1000:.0f} km"
            if c is None:
                continue
            c_type = f"{cref.type}_{'bp' if cref.is_bp else 'sp'}"
            comp_label = f"{c_type}_{cref.cut:.0f}_cutoff_dist"
            if cref.decay_from > -1:
                comp_label = f"{c_type}_{cref.cut:.0f}_decay"
            comp_variant_label = f"{c_type} {c_label} : {cref_label}"
            # run comparison
            centr_summary.append(
                h.centr_comparison(aoi, centr_df, c, cref, dir_detail_plot=dir_detail_plot, centr_diff_name=comp_label,
                                   c_label=c_label, cref_label=cref_label, ccomp_label=comp_variant_label, generate_plots=generate_plots)
                )
            #centr_summary.append({"test":"x", "cl": c_label, "crefl": cref_label, "ctype": c_type, "c": str(c), "cref": str(cref)})

In [ ]:
if not recalc and os.path.exists(f_diffstats_dist_cutoff):
    cdf = pd.read_csv(f_diffstats_dist_cutoff)
    display(cdf.head())
else:
    centr_summary = []
    for aoi in aoi_names:
        run_comparison(aoi, centr_summary)
        
    cdf = pd.DataFrame.from_dict(centr_summary, orient="columns")
    display(cdf.head())
    cdf.to_csv(f_diffstats_dist_cutoff) 

In [ ]:
cdf

In [ ]:
grp = cdf.loc[:,["label_compare", "d_max_c", "d_max_c_rel", "d_hc_seg_n", "d_hc_seg_n_rel",
           "hc_seg_added", "hc_seg_removed", "hc_seg_remained",
           "hc_seg_share_changed", "hcq_d_mean", "hcq_dn_mean", "hcq_dn_rel_mean",
           "hcp_d_mean", "hcp_dn_mean", "hcp_dn_rel_mean",
           "dn_incr_gt10", "dn_decr_gt10", 
           'dn_incr_gt20', 'dn_decr_gt20', 'n_turned_zero']].groupby(["label_compare"])
mean_agg = grp.mean()
mean_agg

In [ ]:
# TODO: sep. charts for bp and sp variants, sort by values (ranked)

mean_agg["hc_seg_share_changed"].plot.bar()

In [ ]:
mean_agg["hcq_dn_mean"].plot.bar()

In [ ]:
mean_agg["hcp_dn_mean"].plot.bar()

### bikeable routes

In [ ]:
g = cdf[cdf.c_is_bp].groupby("label_compare")
g.hc_seg_share_changed.describe()

In [ ]:
plot = g.hc_seg_share_changed.describe().sort_values(by="mean")[["min", "mean", "max"]].plot(
    figsize=(10,3), color=[P_C_MIN, P_C_MEAN, P_C_MAX], xlabel="sbc bp variants: dist decay / cutoff [km]")
plt.setp(plot.axes.get_xticklabels(True), visible=True)
plt.setp(plot.axes.get_xticklabels(False), visible=True)
tls = g.hc_seg_share_changed.describe().sort_values(by="mean")[["min", "mean", "max"]].index.values
tls_renamed = []
for l in tls:
    tls_renamed.append(l.lstrip("sbc_bp ").replace(" km", ""))
plt.xticks([i for i in range(len(tls))],tls_renamed)
h.save_plot("hc_share_chg_all", plot_dir, show=True)